# Автоматизация поддержки клиентов с несколькими агентами

В этом уроке вы узнаете о шести ключевых элементах, которые помогают агентам работать еще эффективнее:
- Ролевые игры
- Фокус
- Инструменты
- Взаимодействие
- Ограничения
- Память

In [37]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [38]:
from crewai import Agent, Task, Crew
from langchain.globals import set_llm_cache
set_llm_cache(None)

In [39]:
import os
from dotenv import load_dotenv
load_dotenv() 

print(os.environ["OPENAI_MODEL_NAME"], " at ", os.environ["OPENAI_API_BASE"])

gpt-4.1-nano  at  https://api.proxyapi.ru/openai/v1


## Ролевая игра, фокус и взаимодействие

In [40]:
support_agent = Agent(
    role="Старший специалист службы поддержки",
	goal="Быть самым дружелюбным и полезным специалистом службы поддержки в вашей команде",
	backstory=(
		"Вы работаете в crewAI (https://crewai.com) и сейчас занимаетесь предоставлением поддержки {customer}, "
        "очень важному клиенту вашей компании. Вам нужно убедиться, что вы предоставляете лучшую поддержку! "
        "Обязательно давайте полные, исчерпывающие ответы и не делайте предположений."
	),
    allow_delegation=False,
	verbose=True
)

- Не устанавливая `allow_delegation=False`, `allow_delegation` принимает значение по умолчанию — `True`.
- Это означает, что агент _может_ передать свою работу другому агенту, который лучше подходит для выполнения конкретной задачи.

In [41]:
support_quality_assurance_agent = Agent(
	role="Специалист по обеспечению качества поддержки",
	goal="Получить признание за предоставление лучшего обеспечения качества поддержки в вашей команде",
	backstory=(
		"Вы работаете в crewAI (https://crewai.com) и сейчас взаимодействуете с вашей командой по запросу {customer}, "
        "чтобы убедиться, что специалист службы поддержки предоставляет максимально качественную поддержку. "
        "Вам нужно удостовериться, что специалист службы поддержки даёт полные, исчерпывающие ответы "
        "и не делает предположений."
	),
    verbose=True
)

* **Ролевая игра**: Обоим агентам назначены роли, цели и предыстория.  
* **Фокус**: Обоим агентам поручено войти в роль своих персонажей.  
* **Взаимодействие**: Агент по обеспечению качества поддержки может делегировать работу обратно агенту поддержки, позволяя этим агентам совместно работать.

## Инструменты, ограничения и память

### Инструменты

In [42]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

### Возможные пользовательские инструменты
- Загрузка данных клиента
- Получение доступа к предыдущим беседам
- Загрузка данных из CRM
- Проверка существующих отчетов об ошибках
- Проверка существующих запросов на добавление функций
- Проверка текущих заявок
- ... и многое другое

- Создайте экземпляр инструмента для извлечения документов.  
- Инструмент скрапит страницу (только один URL) документации CrewAI.

In [43]:
docs_scrape_tool = ScrapeWebsiteTool(website_url='https://docs.crewai.com/concepts/memory'
    #website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

In [44]:
#docs_scrape_tool.run()

##### Различные способы предоставления агентам инструментов

- Уровень агента: агент может использовать инструменты в любой задаче, которую он выполняет.  
- Уровень задачи: агент будет использовать инструменты только при выполнении конкретной задачи.

**Примечание**: Инструменты задачи переопределяют инструменты агента.

### Создание задач
- Вы передаёте инструмент на уровне задачи.

In [45]:
inquiry_resolution = Task(
    description=(
        "{customer} только что обратился с очень важным запросом:\n {inquiry}. \n"
        "Обратился {person} из {customer}. Обязательно используйте всю доступную информацию, "
        "чтобы предоставить максимально качественную поддержку. Вы должны стараться дать полный "
        "и точный ответ на полученный запрос. При необходимости можно посмотреть документацию на сайте."
    ),
    expected_output=(
	    "Подробный, информативный ответ на запрос клиента, который охватывает все аспекты его вопроса. "
        "Ответ должен включать ссылки на все использованные источники для поиска ответа, включая внешние данные или решения. "
        "Ответ должен быть полным, чтобы не осталось ни одного вопроса без ответа, "
        "и поддерживайте дружелюбный и готовый помочь тон на протяжении всего общения."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

- `quality_assurance_review` не использует никаких инструментов.  
- Здесь агент по обеспечению качества только проверяет работу агента поддержки.

In [46]:
quality_assurance_review = Task(
    description=(
        "Проверьте ответ, подготовленный старшим специалистом поддержки на запрос {customer}. "
        "Убедитесь, что ответ является полным, точным и соответствует высоким стандартам качества поддержки клиентов. "
        "Проверьте, что все части вопроса клиента были подробно рассмотрены, в помогающем и дружелюбном тоне. "
        "Проверьте наличие ссылок и источников, использованных для поиска информации, чтобы убедиться, "
        "что ответ хорошо обоснован и не оставляет без ответа ни одного вопроса."
        "Обратите внимание, ответ должен быть на русском языке! Если что-то не так, надо указать на ошибки и попросить изменить."
    ),
    expected_output=(
        "Готовый окончательный, подробный и информативный ответ, который можно отправить клиенту. "
        "Этот ответ должен полностью охватывать запрос клиента, учитывать предоставленную обратную связь "
        "и включать все соответствующие правки и улучшения. Текст должен быть на русском языке."
        "Не будьте слишком формальными — мы дружелюбная и классная компания, "
        "но сохраняйте профессиональный и приятный тон на протяжении всего сообщения."
    ),
    agent=support_quality_assurance_agent,
)


### Создание команды

#### Память
- Установка `memory=True` при формировании команды включает использование памяти.

In [47]:
embedder_config = {
    "provider": "openai",
    "config": {
        "api_key": os.getenv("OPENAI_API_KEY"),
        "api_base": os.getenv("OPENAI_API_BASE"),
        "model": "text-embedding-3-small",
    }
}

In [48]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=True,
  memory=True,
  embedder=embedder_config
)

### Запуск команды

**Примечание**: Модели языковых моделей могут выдавать разные результаты при одинаковом вводе, поэтому то, что вы получите, может отличаться от того, что видно на видео.

#### Ограничения
- Запустив приведённый ниже пример выполнения, вы сможете убедиться, что агенты и их ответы соответствуют ожидаемому поведению.

In [49]:
inputs = {
    "customer": "Газпром",
    "person": "Алексей Миллер",
    "inquiry":  "Мне нужна помощь в создании команды (Crew) и её запуске. "
                "Конкретно, интересует, как добавить память (Memory) в мою команду (Crew). "
                "Можете подсказать?"
}
result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 272bc50d-ff7f-49ee-aa16-4599d07f9104                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 0ededf38-ea77-4cd8-aaba-f0db8ea11f88
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 0ededf38-ea77-4cd8-aaba-f0db8ea11f88
       Status: Executing Task...
    └── 🤖 Agent: Старший специалист службы поддержки
            Status: In Progress

# Agent: Старший специалист службы поддержки
## Task: Газпром только что обратился с очень важным запросом:
 Мне нужна помощь в создании команды (Crew) и её запуске. Конкретно, интересует, как добавить память (Memory) в мою команду (Crew). Можете подсказать?. 
Обратился Алексей Миллер из Газпром. Обязательно используйте всю доступную информацию, чтобы предоставить максимально качественную поддержку. Вы должны стараться дать полный и точный ответ на полученный запрос. При необходимости можно посмотреть документацию на сайте.


# Agent: Старший специалист службы поддержки
## Final Answer: 
Thought: Чтобы предоставить максимально точную и полную инструкцию по добавлению памяти в команду (Crew) в системе crewAI, необходимо ознакомиться с официальной документацией по управлению памятью. Для этого я воспользуюсь инструментом "Read website content" по ссылке https://docs.crewai.com/concepts/memory, чтобы изучить все актуальные сведения и рекомендации. После получения информации я подготовлю р

🚀 Crew: crew
└── 📋 Task: 0ededf38-ea77-4cd8-aaba-f0db8ea11f88
       Status: Executing Task...
    └── 🤖 Agent: Старший специалист службы поддержки
            Status: ✅ Completed

...

# Agent: Специалист по обеспечению качества поддержки
## Task: Проверьте ответ, подготовленный старшим специалистом поддержки на запрос Газпром. Убедитесь, что ответ является полным, точным и соответствует высоким стандартам качества поддержки клиентов. Проверьте, что все части вопроса клиента были подробно рассмотрены, в помогающем и дружелюбном тоне. Проверьте наличие ссылок и источников, использованных для поиска информации, чтобы убедиться, что ответ хорошо обоснован и не оставляет без ответа ни одного вопроса.Обратите внимание, ответ должен быть на русском языке! Если что-то не так, надо указать на ошибки и попросить изменить.


# Agent: Специалист по обеспечению качества поддержки
## Final Answer: 
Read website content




🚀 Crew: crew
├── 📋 Task: 0ededf38-ea77-4cd8-aaba-f0db8ea11f88
│      Assigned to: Старший специалист службы поддержки
│      Status: ✅ Completed
│   └── 🤖 Agent: Старший специалист службы поддержки
│           Status: ✅ Completed
└── 📋 Task: a6710b52-774d-46fd-a8b6-3e5b3f207956
       Status: Executing Task...
    └── 🤖 Agent: Специалист по обеспечению качества поддержки
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 0ededf38-ea77-4cd8-aaba-f0db8ea11f88
│      Assigned to: Старший специалист службы поддержки
│      Status: ✅ Completed
│   └── 🤖 Agent: Старший специалист службы поддержки
│           Status: ✅ Completed
└── 📋 Task: a6710b52-774d-46fd-a8b6-3e5b3f207956
       Assigned to: Специалист по обеспечению качества поддержки
       Status: ✅ Completed
    └── 🤖 Agent: Специалист по обеспечению качества поддержки
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: a6710b52-774d-46fd-a8b6-3e5b3f207956                                                                     │
│  Agent: Специалист по обеспечению качества поддержки                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 272bc50d-ff7f-49ee-aa16-4599d07f9104                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

- Отобразите окончательный результат в виде Markdown.

In [50]:
from IPython.display import Markdown
Markdown(result.raw)

Read website content